<a href="https://colab.research.google.com/github/Soham-Shah0107/Course-Allocation/blob/main/Fair_Allocation_SPIRE_and_Round_Robin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
import numpy as np

Reading Excel File into Pandas DataFrame
Splitting Time Column into Start_Time and End_Time

In [ ]:
import pandas as pd
df = pd.read_excel('/course_schedule.xlsx')
dic = {}
for row in df.iterrows():
    if(isinstance(row[1][1],str)):
        if("#" not in row[1][1]):
            classes = row[1][1]
        else:
            if classes in dic.keys():
                dic[classes].append(row[1][1])
            else:
                dic[classes] = [row[1][1]]
df[["Start_Time","End_Time"]] = df.TIME.str.split("-",expand = True)
print(df.head())

In [ ]:
import re
df["DAYS"] = (df["DAYS"].str.split(r'(?=[A-Z])'))
df["DAYS"] = df["DAYS"].apply(lambda x: x[1:] if isinstance(x, list) else x)


In [ ]:
'''
{
  "class1":"time",
  "class2":"time",
  "class3":"time2"
}
'''
schedule = {}
for index,row in df.iterrows():
    if(isinstance(row[1],str)):
        if("#" in row[1]):
            if(isinstance(row[2],list)):
                schedule[row["CLASS#"]] = str(row[2]) +  str(row[3])
print(schedule.values())

In [ ]:
#create a test schedule preference
testPref = [0 for x in range(len(schedule))]
testPref[0] = 1
testPref[1] = 1


In [ ]:
# Number of agents
n = 3

# Number of goods
m = 6

priority = [1,2,3] #priority order, lowest goes first

weights = [1, 1, 1] #importance weighting
cap = [2,2,2,2] # classes cap per student

preferences = [
    # You can modify the preferences of Anjali here
    [1, 1, 1, 1, 0, 1],
    # You can modify the preferences of Bob here
    [1, 0, 1, 0, 1, 1],
    # You can modify the preferences of Cai here
    [1, 1, 0, 0, 1, 1],

    #test pref
    testPref
]
#slots = {0:0, 1:0, 2:1, 3:1, 4:2, 5:3} #time slot for courses
slots = list(schedule.values()) #

In [ ]:
def SpireAlloc(n, m, cap, preferences, priority):
  X = [[0]*m for i in range(n)] # Initialize the allocation as a binary matrix (n x m)
  playerInd = list(range(n)) # list of player indicies to iterate through
  playerInd = [x for _, x in sorted(zip(priority, playerInd))] #reorder players by priority (lowest goes first)

  availClasses = [1 for i in range(m)] #list of available classes 

  for i in playerInd:
    #add each class in the preference if available
    curClasses = 0
    for j in range(len(preferences[i])):
      if (availClasses[j] > 0 and preferences[i][j] == 1):
        #print(j,preferences[i][j],availClasses[j])
        availClasses[j] -= 1 #remove 1 class from avail
        X[i][j] = 1
        curClasses += 1
      if curClasses >= cap[i]: #stop adding classes after cap
        break
    print(f"Player {i} preferred:", preferences[i])
    print(f"Player {i} classes  :", X[i])
    print("-----")
  return X

In [ ]:
SpireAlloc(n, m, cap, preferences, priority)

In [ ]:
# My understanding of Red Robin: Each student takes a turn getting 1 class, until every single student has reached their cap
#WIP
def RedRobin(cap, spots, preferences, priority): #cap: num[n], spots: num, preferences: num[n][m], priority: num[n]
  n = len(preferences) #number of students
  m = len(preferences[0]) #number of classes

  X = [[0]*m for i in range(n)] # Initialize the allocation as a binary matrix (n x m)
  playerInd = list(range(n)) # list of player indicies to iterate through
  playerInd = [x for _, x in sorted(zip(priority, playerInd))] #reorder players by priority (lowest goes first)

  availClasses = [spots for i in range(m)] #list of available classes 

  while availClasses.count(0) < len(availClasses): #loop as long as there are classes left
    #go through a full cycle
    for i in playerInd:
      #add each class in the preference if available
      curClasses = X[i].count(1) #count number of filled slots
      if curClasses >= cap[i]: #stop adding classes after cap
        continue
      for j in range(len(preferences[i])):
        if (X[i][j] != 1 and availClasses[j] > 0 and preferences[i][j] == 1): #must not already own class, class must be available, must be a preference
          availClasses[j] -= 1 
          X[i][j] = 1
          break
      
  #print results
  for i in playerInd:
    print(f"Player {i} preferred:", preferences[i])
    print(f"Player {i} classes  :", X[i])
    print("-----")
  return X

In [ ]:
RedRobin(cap, preferences, priority)

In [ ]:
#Testing for ROUND Robin 
prefs = [[1,0,0,1],
     [1,1,0,0],
     [0,1,1,0],
     [1,0,1,1]]
cap = 2
RedRobin([cap,cap,cap,cap],2,prefs, [1,2,3,4]) #doesn't work if there are leftover spots!!!!! FIX THIS TODO

Player 0 preferred: [1, 0, 0, 1]
Player 0 classes  : [1, 0, 0, 1]
-----
Player 1 preferred: [1, 1, 0, 0]
Player 1 classes  : [1, 1, 0, 0]
-----
Player 2 preferred: [0, 1, 1, 0]
Player 2 classes  : [0, 1, 1, 0]
-----
Player 3 preferred: [1, 0, 1, 1]
Player 3 classes  : [0, 0, 1, 1]
-----


[[1, 0, 0, 1], [1, 1, 0, 0], [0, 1, 1, 0], [0, 0, 1, 1]]

In [ ]:
from IPython.core.display import Math
''' Max Matching Algorithm 
Grabs preferences for agent i from "preferences" var
Grabs time slot conflict from "slots" var
S is in the same format as preferences[i]
'''
def maxMatch(i, S): #agent i, given classes S
  B = [0 for n in range(len(S))] #Best class selection
  for c in range(len(S)):
    valid = True
    
    if not (S[c] == 1 and preferences[i][c] == 1): #skip if S is not assigned or is not a preference
      valid = False
    for j in range(len(B)):
      if B[j] == 1 and slots[j] == slots[c]: #skip if time slot matches a previously existing time slot
        valid = False
    
    if valid:
      B[c] = 1 #add the class
  return min(B.count(1), cap[i]) #return number of classes in B, but capped

maxMatch(3, testPref)

In [ ]:
import random
#generate students (class preferences) based on related topics and professor
'''
Given:
a list of classes a professor teaches 
a list of classes for each topic

'''
#Say classes are as follows: Algorithms by Alice, AI by Alice, Data Structures by David, ML by Mark, 
topics = {"basics":[0,2], "ML":[1,3]}
professors = {"Alice":[0,1],"David":[2],"Mark":[3]}

def genStudent(m, cap, tops, profs): #m: num, cap: num, tops: str[], profs: str[]
  S = [0 for n in range(m)] #class selection
  L = [] #list of possible wanted classes

  #add all classes from a preferred topic or professor
  for i in tops:
    for j in topics[i]:
      L.append(j)
  for i in profs:
    for j in professors[i]:
      L.append(j)
  #note - if there is a overlap between a pref. professor/topic, double count the class (higher weight for random selection)
  #TODO: deduplicate results
  
  #random selection
  print("--")
  print(L)
  random.shuffle(L)
  for i in range(min(cap,len(L))): 
    S[L[i]] = 1 #pick random classes from list of classes until cap is reached
  return S

#some tests
print(genStudent(4,2,[],["Alice"]))
print(genStudent(4,2,[],["Alice","David"]))
print(genStudent(4,2,["basics"],[]))
print(genStudent(4,2,["basics"],["Alice"]))
print(genStudent(4,2,["ML"],["Alice","David"]))

--
[0, 1]
[1, 1, 0, 0]
--
[0, 1, 2]
[1, 1, 0, 0]
--
[0, 2]
[1, 0, 1, 0]
--
[0, 2, 0, 1]
[1, 1, 0, 0]
--
[1, 3, 0, 1, 2]
[0, 1, 1, 0]


IndexError: ignored